## See the ReadME for a description of the project


### Importing dependencies

In [10]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

### Load Data

In [11]:
# define number of classes
n_classes = 10

# load train and test sets
train = pd.read_csv('Data/fashion-mnist_train.csv')
test = pd.read_csv('Data/fashion-mnist_test.csv')

train = train.to_numpy()
test = test.to_numpy()

X_train = train[:,1:]
y_train = train[:,0]
X_test = test[:,1:]
y_test = test[:,0]
print("Train data shape:", X_train.shape, "Test data shape:", X_test.shape)
print("Train labels shape:", y_train.shape,"  Test labels shape:", y_test.shape)

Train data shape: (60000, 784) Test data shape: (10000, 784)
Train labels shape: (60000,)   Test labels shape: (10000,)


In [12]:
# this section is inspired by code from the article:
# "Build an Image Classification Model using Convolutional Neural Networks in PyTorch"
# Author: Pulkit Sharma
# https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/

# change training and test set into tensors
X_train = X_train.reshape(len(X_train),1,28,28)
X_train = torch.from_numpy(X_train)
X_train = X_train.type(torch.FloatTensor)

y_train = y_train.astype(int)
y_train = torch.from_numpy(y_train)
y_train = y_train.type(torch.LongTensor)

X_test = X_test.reshape(len(X_test),1,28,28)
X_test = torch.from_numpy(X_test)
X_test = X_test.type(torch.FloatTensor)

y_test = y_test.astype(int)
y_test = torch.from_numpy(y_test)
y_test = y_test.type(torch.LongTensor)

# verify shape of training data
print("Train data shape:", X_train.shape, "Test data shape:", X_test.shape)
print("Train labels shape:", y_train.shape,"  Test labels shape:", y_test.shape)




Train data shape: torch.Size([60000, 1, 28, 28]) Test data shape: torch.Size([10000, 1, 28, 28])
Train labels shape: torch.Size([60000])   Test labels shape: torch.Size([10000])


## Implementing the CNN model

In [13]:
# this section is inspired by code from the article:
# "Build an Image Classification Model using Convolutional Neural Networks in PyTorch"
# Author: Pulkit Sharma
# https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/


class Net(Module):
    def __init__(self, kernel_size = 2, stride = 1, padding = 1):
        super(Net,self).__init__()
        
        self.cnn_layers = Sequential(
            # Define convolutional layers
            # input dimensionality before each layer is added
            
            # 1st layer
            # 1x28x28
            Conv2d(1,4,kernel_size=kernel_size,stride=stride,padding=padding),
            # apply batch normalization
            #BatchNorm2d(4),
            # activation function ReLU
            ReLU(),
            # apply pooling
            MaxPool2d(kernel_size=kernel_size,stride=stride),
            
            # second layer
            # 4x14x14
            Conv2d(4,4,kernel_size=kernel_size,stride=stride,padding=padding),
            #BatchNorm2d(4),
            ReLU(),
            MaxPool2d(kernel_size=kernel_size,stride=stride),
            
            # 4x7x7
        )
        
        self.linear_layers = Sequential(
            Linear(4*7*7,n_classes)
        )
    
    def update_linear_layers(self,input_size):
        self.linear_layers = Sequential(
            Linear(input_size,n_classes)
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0),-1)
        self.update_linear_layers(x.size()[1])
        x = self.linear_layers(x)
        return x

In [14]:
def train(epoch, model, optimizer, train_losses, val_losses, criterion):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train_t, y_train_t = Variable(X_train), Variable(y_train)
    # getting the validation set
    x_val_t, y_val_t = Variable(X_test), Variable(y_test)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train_t = x_train_t.cuda()
        y_train_t = y_train_t.cuda()
        x_val_t = x_val_t.cuda()
        y_val_t = y_val_t.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train_t)
    output_val = model(x_val_t)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train_t)
    loss_val = criterion(output_val, y_val_t)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

    return train_losses, val_losses, optimizer, model

In [16]:

def CNN_model_train(kernel_size = 2, stride = 1, padding = 1, learning_rate = 0.07, n_epochs = 5):
    """Define network and 

    Args:
        kernel_size (int, optional): _description_. Defaults to 2.
        stride (int, optional): _description_. Defaults to 1.
        padding (int, optional): _description_. Defaults to 1.
        learning_rate (float, optional): _description_. Defaults to 0.07.
        n_epochs (int, optional): _description_. Defaults to 5.
    """
    # define model
    model = Net(kernel_size, stride, padding)
    # defining the optimizer
    optimizer = Adam(model.parameters(), lr=0.07)
    # defining the loss function
    criterion = CrossEntropyLoss()
    # checking if GPU is available
    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
        
    # defining the number of epochs
    # n_epochs = 5
    # empty list to store training losses
    train_losses = []
    # empty list to store validation losses
    val_losses = []
    # training the model
    for epoch in range(n_epochs):
        train_losses, val_losses, optimizer, model = train(epoch, model, optimizer, train_losses, val_losses, criterion)
    print(model)

In [17]:
CNN_model_train()

Epoch :  1 	 loss : tensor(29.7087, grad_fn=<NllLossBackward>)


## 